In [1]:
#pip install transformers trl accelerate torch bitsandbytes peft datasets -qU
#pip install scipy

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
)

In [3]:
MODEL_PATH = f"../models/openchat-3.5-1210"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=nf4_config,
    device_map='auto',
    local_files_only=True,
    trust_remote_code=False,
    use_cache=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
deci_generator = pipeline("text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          temperature=0.1,
                          device_map="auto",
                          max_length=1024,
                          return_full_text=False
)

In [23]:
def ask(user_input):
   #GPT4 Correct User: Hello<|end_of_turn|>GPT4 Correct Assistant: Hi<|end_of_turn|>GPT4 Correct User: How are you today?<|end_of_turn|>GPT4 Correct Assistant:
   SYSTEM_PROMPT_TEMPLATE ="""GPT4 Correct User: {user_input}<|end_of_turn|>GPT4 Correct Assistant: """
   prompt = SYSTEM_PROMPT_TEMPLATE.format(user_input=user_input)
   inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
   output = model.generate(**inputs,
                            max_new_tokens=500,
                            num_beams=5,
                            no_repeat_ngram_size=4,
                            early_stopping=True,
                            eos_token_id=tokenizer.eos_token_id, 
                            pad_token_id=tokenizer.eos_token_id, 
                            )
   output = tokenizer.decode(output[0], skip_special_tokens=False)
   ss = output.split('<|end_of_turn|>')[-2:]
   ss = ''.join(ss).replace('GPT4 Correct Assistant: ', '').lstrip()
   return ss
   
   

In [7]:
ask("What is your name?")

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'My name is ChatGPT, a large language model created by OpenAI.'

In [60]:
def ask_qa(context, user_question):
   prompt = """
Instruction: Answer the question based on following private context. 
The response content must appear within the private context.
If the answer cannot be found from the following private context, respond with 'I don't know.'.
Here is private context to help:

{context}

Here is QUESTION:
{question} 
"""
   return ask(prompt.format(context=context, question=user_question))

In [21]:
context = """
Question: What is Depbon in the promo code?
Answer: Depbon means deposit bonus.

Question: What is ACE in the promo code?
Answer: ACE means MNL Bonus for BET.
South Korea is an exception. South Korea players use the 'TOP' promo code.
"""

answer = ask_qa(context, "What is ACE?")
answer

output='<s> GPT4 Correct User: \nInstruction: Answer the question based on your gaming knowledge. \nIf the answer cannot be found from the context, respond with \'I don\'t know.\'.\nHere is context to help:\n\n\nQuestion: What is Depbon in the promo code?\nAnswer: Depbon means deposit bonus.\n\nQuestion: What is ACE in the promo code?\nAnswer: ACE means MNL Bonus for BET.\nSouth Korea is an exception. South Korea players use the \'TOP\' promo code.\n\n\nUser QUESTION:\nWhat is ACE? \n<|end_of_turn|> GPT4 Correct Assistant: \nACE in a promo code typically stands for a bonus or promotion offered by an online gaming platform. In some cases, it may mean "MNL Bonus" for BET, while in South Korea, it may refer to the "TOP" promo code for players. The specific meaning may vary depending on the platform and region.<|end_of_turn|>'


'ACE in a promo code typically stands for a bonus or promotion offered by an online gaming platform. In some cases, it may mean "MNL Bonus" for BET, while in South Korea, it may refer to the "TOP" promo code for players. The specific meaning may vary depending on the platform and region.'

In [31]:
def read_file(file):
   with open(file, 'r', encoding='utf-8') as f:
      return f.read()

In [64]:
context = read_file('../fine-tune/data/Test.md')

In [48]:
answer = ask_qa(context, "Introduce Baccarat")
answer

'Baccatat is a card game that is popular in casinos and involves players betting on the outcome of a hand between the Player and the Banker. The goal is to get a hand value as close to 9 as possible, with the use of specific rules determining if a third card is needed. The game is known for its straightforward rules and enticing atmosphere.'

In [51]:
answer = ask_qa(context, "What is error code E8000?")
answer

"# Error Code Table\nError Code | Description\n---|----\nE1000 | Unknown error\nE2001 | Connection error\nE3002 | Authentication error\nE4003 | Authorization error\nE5004 | Data error\nE6005 | System error\nE7006 | Interface error\nE8007 | Side bet error\nE9008 | Game rule error\nEA009 | Audio error\nEB00A | Video error\nEC00B | Graphic error\nED00C | Input error\nEE00D | Output error\nEF00E | Display error\nEG00F | Environment error\nEH00G | Hardware error\nEI00H | Software error\nEJ00I | Communication error\nEK00J | Network error\nEL00K | Location error\nEM00L | Language error\nEN00M | Currency error\nEO00N | Time error\nEP00O | Odds error\nEQ00P | Privacy error\nER00Q | Regulation error\nES00R | Security error\nET00S | Troubleshooting error\nEU00T | Update error\nEV00U | User error\nEW00V | Warranty error\nEX00X | Miscellaneous error\nEY00Y | Custom error\nEZ00Z | Reserved error\n\nBased on the private context provided, I don't have enough information to answer the question about er

In [55]:
answer = ask_qa(context, "Summarize the content.")
answer

'The private context discusses odds tables for two games, SEC Game and MAYA game. The SEC Game odds table includes three entries with their respective average scores, while the MAYA Games odds table lists the same three entries with the same average scores. The context also introduces the game of Baccatat, explaining its basic rules and its popularity among players. Finally, it introduces Tiger Bacarrat as a modification of the original game, offering additional side bets for players.'

In [63]:
answer = ask_qa(context, "What is odds of MAYA Game?")
answer

'The odds table for the MAYA game is as follows:\n\n1. Mistral‑T5‑7B‑v1 has an odds of 12.\n2. openchat‑3.5‑12.10 has odds of 35.\n3. mistral‑7b‑instruct‑v.0.2 has a odds of 57.'

In [65]:
answer = ask_qa(context, "what is percentage Table of SEC Game?")
answer

'The percentage Table of the SEC Game shows the average percentage for each player in the game. Based on the provided private context, the percentages are as follows:\n\n- Player A has an average of 67.1%\n- Player B has a 59.2% average\n- Player C has a 49.3% average'

In [ ]:
answer = ask_qa(context, "Generate several questions and answers based on the content, with each answer contained within the private context. The cumulative content of all answers must cover over 90% of the private context.")
answer